In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time,os
from PIL import Image
from io import BytesIO
import itertools
import pyautogui

# Interval between mouse movements (in seconds)
movement_interval = 60  # Move the mouse every 60 seconds

us_cities = [
    ('Houston','IAH'),
    ('Dallas','DFW'),
    ('Austin','AUS'),
    ('San Francisco', 'SFO'),
    ('Los Angeles','LAX'),
    ('Chicago','ORD'),
    ('New York','EWR'),
    ('New York','JFK'),
    ('Seattle','SEA'),
    ('Vancouver','YVR')
]

asia_cities = [
    ('Hong Kong','HKG'),
    ('Guangzhou','CAN'),
    ('Shanghai','PVG'),
    ('Shanghai','SHA'),
    ('Shenzhen', 'SZX'),
    ('Seoul','ICN'),
    ('Tokyo','NRT'),
    ('Tokyo','HND'),
    ('Beijing','PEK')
]

date_leave = [
   '2023-12-14',
   '2023-12-15',
   '2023-12-16',
   '2023-12-17',
   '2023-12-18',
   '2023-12-19',
   '2023-12-20',
   '2023-12-21',
   '2023-12-22',
   '2023-12-23',
   '2023-12-24',
   '2023-12-25',
   '2023-12-26',
   '2023-12-27',
   '2023-12-28',
]

date_comeback = [
    '2023-12-31',
    '2024-01-11',
    '2024-01-07',
    '2024-01-06',
    '2024-01-08',
    '2024-01-12',
    '2024-01-13',
    '2024-01-14',
    '2024-01-15',
]

# cabin_class = ['business','economy']
cabin_class = ['business']

def keep_screen_active()->None:
    # Get the current mouse position
    current_x, current_y = pyautogui.position()

    # Simulate moving the mouse cursor slightly (1 pixel) to keep the screen active
    pyautogui.moveTo(current_x + 3, current_y + 3)

    # Pause for the specified interval
    # pyautogui.sleep(movement_interval)

def link_builder(departure:str,arrival:str,class_of_service:str,departure_date:str)->str:
    # return f'https://www.point.me/results?departureCity={departure[0]}&departureIata={departure[1]}&arrivalCity={arrival[0]}&arrivalIata={arrival[1]}&legType=oneWay&classOfService={class_of_service}&passengers=1&pid=&departureDate={departure_date}T06%3A00%3A00.000Z'
    return f'https://www.pointsyeah.com/search?airlineProgram=&arrival={arrival[1]}&bankpromotion=false&banks=&cabin=Business%20%26%20First&departDate={departure_date}&departDateSec={departure_date}&departure={departure[1]}&multiday=false&passenger=1&pointpromotion=false&tripType=1'
    
def wait_for_results() -> None:
    time.sleep(25)
    # while True:
    #     try:
    #         driver.find_element(By.XPATH,"//button[normalize-space()='Minimize']")
    #         time.sleep(5)
    #     except:
    #         while True:
    #             try:
    #                 driver.find_element(By.XPATH,"//button[normalize-space()='Maximize']")
    #             except:
    #                 break
    #         break


def click_to_sort_lowest_first()-> bool:
    try:
        sort_button = driver.find_element(By.CSS_SELECTOR,"div[id='headlessui-menu-button-:rg:'] div:nth-child(2)")
        sort_button.click()
        lowest_button = driver.find_element(By.XPATH,"//div[normalize-space()='Points Low to High']")
        lowest_button.click()
        return True
    except:
        return False


def screenshot(path:str)->None:
    # # Get the total height of the page by executing JavaScript in the browser
    # total_height = driver.execute_script("return document.body.scrollHeight")

    # # Set the browser window size to cover the entire page
    # driver.set_window_size(1920, total_height)

    # # Initialize the final screenshot as a blank image with the same width as the browser window
    # screenshot = Image.new('RGB', (1920, total_height))


    # flight_info = driver.find_element(By.CSS_SELECTOR,"body > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)")


    # screenshot.paste(Image.open(BytesIO(flight_info.screenshot_as_png)), (0, 0))

    # current_height = flight_info.size['height']
    # flight_element = driver.find_element(By.CSS_SELECTOR,"body > div:nth-child(2) > div:nth-child(2) > div:nth-child(3) > div:nth-child(3)")
    # i = 0
    # while True:
    #     try:
    #         flight = flight_element.find_element(By.CSS_SELECTOR,"div[i='{}']".format(i))
    #         driver.implicitly_wait(0.5)
    #         # flight.screenshot('/Users/xianggao/Downloads/flight_{}.png'.format(i))
    #         screenshot.paste(Image.open(BytesIO(flight.screenshot_as_png)), (0, current_height))
    #         current_height+=flight.size['height']
    #     except:
    #         break
    #     finally:
    #         pass
    #     i+=1
    #     if i>5:
    #         break
    # screenshot = screenshot.crop((0, 0, flight_info.size['width'], current_height))
    # screenshot.save(path)
    screenshot = driver.get_screenshot_as_png()

    # Save the screenshot to a file
    with open(path, "wb") as file:
        file.write(screenshot)

    # # Open the screenshot using Pillow (PIL)
    # img = Image.open("screenshot.png")
    # img.show()

os.system('pkill Goole Chrome')

# Get the current time as a struct_time object
current_time_struct = time.localtime()

# Format the current time as a string using strftime
formatted_time = time.strftime("%Y_%m_%d_%H_%M_%S", current_time_struct)

results_path = f"/Users/xianggao/Downloads/flight/{formatted_time}"
os.mkdir(results_path)

options = webdriver.ChromeOptions()
options.add_argument("--verbose")
# options.add_argument('--headless')
options.add_argument("user-data-dir=/Users/xianggao/Downloads/chrome_profile/Default")
driver = webdriver.Chrome(options=options)

for depature,arrival,date,class_of_service in itertools.product(asia_cities,us_cities,date_comeback,cabin_class):
    keep_screen_active()
    screenshot_path = f'{results_path}/{depature[1]}_{arrival[1]}_{date}_{class_of_service}.png'
    driver.get(link_builder(depature,arrival,class_of_service,date))
    wait_for_results()
    # if click_to_sort_lowest_first():
    screenshot(screenshot_path)
 
for depature,arrival,date,class_of_service in itertools.product(us_cities,asia_cities,date_leave,cabin_class):
    keep_screen_active()
    screenshot_path = f'{results_path}/{depature[1]}_{arrival[1]}_{date}_{class_of_service}.png'
    driver.get(link_builder(depature,arrival,class_of_service,date))
    wait_for_results()
    # if click_to_sort_lowest_first():
    screenshot(screenshot_path)

# # Load the website
# driver.get('https://www.point.me/results?departureCity=Austin&departureIata=AUS&arrivalCity=Hong+Kong&arrivalIata=HKG&legType=oneWay&classOfService=business&passengers=1&pid=&departureDate=2023-12-23T06%3A00%3A00.000Z')

KeyboardInterrupt: 